In [167]:
import requests
from bs4 import BeautifulSoup

import bs4
import pandas as pd
import numpy as np
import time

In [168]:
URL = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm" 

page = requests.get(URL)
#specifying the desired format of "page" using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, "html.parser")
#printing soup in a more structured tree format that makes for easier reading
print(soup.prettify())

<html>
 <head>
  <title>
   403 Forbidden
  </title>
 </head>
 <body>
  <center>
   <h1>
    403 Forbidden
   </h1>
  </center>
 </body>
</html>



In [169]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


url = "https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm"

TIMEOUT = 7

options = Options()
options.add_argument("--start-maximized")  # Maximize the browser window
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
options.add_experimental_option("prefs", 
                                {"profile.default_content_setting_values.notifications": 2 
                                }) 

print(f"Retrieving web page URL '{url}'")
driver = webdriver.Chrome(
    # WEB_DRIVER_LOCATION, 
    options=options)
driver.get(url)

# Timeout needed for Web page to render 
time.sleep(TIMEOUT)

html = driver.page_source

# Get the source of the page and create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')

driver.quit()

Retrieving web page URL 'https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm'


In [170]:
print(soup.prettify())

<html class="scriptsOn" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <script async="" crossorigin="anonymous" src="https://images-na.ssl-images-amazon.com/images/I/31bJewCvY-L.js">
  </script>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <title>
   Most Popular Movies
  </title>
  <meta content="As determined by IMDb Users" data-id="main" name="description"/>
  <meta content="IMDb" property="og:site_name"/>
  <meta content="Most Popular Movies" property="og:title"/>
  <meta content="As determined by IMDb Users" property="og:description"/>
  <meta content="website" property="og:type"/>
  <meta content="https://m.media-amazon.com/images/G/01/imdb/images/social/imdb_logo.png" property="og:image"/>
  <meta content="1000" property="og:image:height"/>
  <meta content="1000" property="og:image:

In [171]:
# Function to get top 100 movie names
def get_movie_names(soup):
    movie_names_elements = soup.find_all('h3')
    movie_names = []

    # Scraping Movie Names for top 100 IMDB movies
    for movie in movie_names_elements:
        text = movie.text.strip()
        if text == 'Fast Times at Ridgemont High':
            # Include the matching element and stop further processing
            movie_names.append(text)
            break
        elif text != 'IMDb Charts' and text != 'You have rated':
            # Exclude unwanted elements
            movie_names.append(text)
            
    return movie_names


In [183]:
# Function to get current and previous ranks of the movies
def get_ranking(soup):    
    # Find all <div> elements with the class 'meter-const-ranking' in the BeautifulSoup object 'soup'.
    ranking_divs = soup.find_all('div', class_='meter-const-ranking')
    
    # Initialize empty lists to store current and previous rankings.
    curr_ranking = []
    prev_ranking = []
    
    # Loop through each 'ranking_div' found.
    for ranking_div in ranking_divs:
        # Extract the text within the 'ranking_div', remove leading/trailing whitespace.
        text = ranking_div.text.strip()
        
        # Split the text using '(' as the delimiter to separate current and previous rankings.
        ranks = text.split('(')
        
        # Append the current ranking (the first part) to the 'curr_ranking' list.
        curr_ranking.append(ranks[0].strip())
        
        # Append the previous ranking (the second part without ')' characters) to the 'prev_ranking' list.
        prev_ranking.append(ranks[1].replace(')', '').strip())
        
    # Return the lists containing current and previous rankings.
    return curr_ranking, prev_ranking

In [184]:
# Function to extract Release year, movie duration, and movie category from a BeautifulSoup object 'soup'.
def get_movie_details(soup):
    # Find all <div> elements with the class 'cli-title-metadata' in the 'soup'.
    metadata_div = soup.find_all('div', class_='cli-title-metadata')

    # Initialize empty lists to store Release year, movie duration, and movie category.
    release_year = []
    movie_duration = []
    movie_category = []

    # Loop through each 'metadata' div found.
    for metadata in metadata_div:
        # Find all <span> elements with the class 'cli-title-metadata-item' within 'metadata'.
        metadata_items = metadata.find_all('span', class_='cli-title-metadata-item')
        
        # Extract and append Release year to the 'release_year' list (if available).
        release_year.append(metadata_items[0].text.strip()) if len(metadata_items) > 0 else None
        
        # Extract and append Movie duration to the 'movie_duration' list (if available), otherwise, append NaN.
        movie_duration.append(metadata_items[1].text.strip()) if len(metadata_items) > 1 else movie_duration.append(np.nan)
        
        # Extract and append Movie category to the 'movie_category' list (if available), otherwise, append NaN.
        movie_category.append(metadata_items[2].text.strip()) if len(metadata_items) > 2 else movie_category.append(np.nan)

    # Return the lists containing Release year, movie duration, and movie category.
    return release_year, movie_duration, movie_category


In [174]:
# Function to extract IMDb ratings from object 'soup'.
def get_imdb(soup):
    # Initialize an empty list to store IMDb ratings.
    imdb_ratings = []

    # Find all <div> elements with the class 'cli-ratings-container'.
    rating_spans = soup.find_all('div', class_='cli-ratings-container')

    # Loop through each 'rating_span' found.
    for rating_span in rating_spans:
        # Find the <span> element with the class 'ipc-rating-star' within 'rating_span'.
        rating = rating_span.find('span', class_='ipc-rating-star')
        
        # Extract and clean IMDb rating from the 'rating' element.
        imdb_rating = rating.text.strip().split('(')[0].strip()
        
        # Append the IMDb rating to the 'imdb_ratings' list.
        imdb_ratings.append(imdb_rating)

    # Return the list containing IMDb ratings.
    return imdb_ratings

In [175]:
# Function to extract URLs for movie pages from a BeautifulSoup object 'soup'.
def get_url(soup):
    # Find all <a> elements with the class 'ipc-title-link-wrapper'.
    items = soup.find_all('a', class_='ipc-title-link-wrapper')
    
    # Initialize an empty list to store the URLs.
    hlinks = []

    # Loop through each 'item' found.
    for item in items:
        # Construct the full URL for the movie page using 'href' attribute and append it to 'hlinks'.
        hlink = "https://www.imdb.com" + item.get('href')
        hlinks.append(hlink)
        
        # Limit the number of URLs to 100 (if more URLs are found).
        hlinks = hlinks[:100]
    
    # Return the list containing movie page URLs.
    return hlinks

In [176]:
# Import the pandas library as 'pd' for data manipulation.
import pandas as pd

# Obtain movie names, current rankings, previous rankings, movie details, IMDb ratings, and URLs using defined functions.
movie_names = get_movie_names(soup)
curr_ranking, prev_ranking = get_ranking(soup)
release_year, movie_duration, movie_category = get_movie_details(soup)
imdb_rating = get_imdb(soup)
url = get_url(soup)

# Initialize an empty list 'result' to store movie data entries.
result = []

# Loop through the data to create individual movie data entries.
for i in range(len(curr_ranking)):
    entry = {
        'Rank': curr_ranking[i],
        'Previous Rank': prev_ranking[i],
        'Movie': movie_names[i],
        'Release Year': release_year[i],
        'Movie Duration': movie_duration[i],
        'Movie Category': movie_category[i],
        'IMDb Rating': imdb_rating[i],
        'Url for Movie': url[i]
    }
    
    # Append each movie data entry to the 'result' list.
    result.append(entry)

# Create a Pandas DataFrame from the 'result' list to organize the data.
df = pd.DataFrame(result)

In [178]:
# Set index as current rank
df = df.set_index('Rank')
df

,Previous Rank,Movie,Release Year,Movie Duration,Movie Category,IMDb Rating,Url for Movie
Rank,,,,,,,
1,3,A Haunting in Venice,2023,1h 43m,PG-13,6.8,https://www.imdb.com/title/tt22687790/?ref_=ch...
2,1,Barbie,2023,1h 54m,PG-13,7.1,https://www.imdb.com/title/tt1517268/?ref_=cht...
3,6,Elemental,2023,1h 41m,PG,7.0,https://www.imdb.com/title/tt15789038/?ref_=ch...
4,1,Talk to Me,2022,1h 35m,R,7.2,https://www.imdb.com/title/tt10638522/?ref_=ch...
5,18,Expend4bles,2023,1h 43m,R,5.2,https://www.imdb.com/title/tt3291150/?ref_=cht...
...,...,...,...,...,...,...,...
96,37,Scream VI,2023,2h 2m,R,6.5,https://www.imdb.com/title/tt17663992/?ref_=ch...
97,20,The Menu,2022,1h 47m,R,7.2,https://www.imdb.com/title/tt9764362/?ref_=cht...
98,81,PAW Patrol: The Mighty Movie,2023,1h 32m,PG,6.0,https://www.imdb.com/title/tt15837338/?ref_=ch...


In [182]:
# Extract the data in csv format
import csv

filename = "Top100IMDB" + ".csv"  # Create a CSV file name

    # Save the data to a CSV file
with open(filename, "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)  # Create a CSV writer
    writer.writerow(['Rank','Previous Rank','Movie','Release Year','Movie Duration','Movie Category',
                     'IMDb Rating','Url for Movie'])
    writer.writerows(result)